emojis and twitter

In [71]:
import twitter

In [78]:
# set these

api = twitter.Api(consumer_key=con_key,
consumer_secret=con_secret,
access_token_key=token_key,
access_token_secret=token_secret)

In [223]:
users = api.GetFriends()
users = [u.name for u in users]

tweets = []
for u in users:
    posts = api.GetUserTimeline(screen_name=user)
    tweets.extend(posts)

In [224]:
len(tweets)

340

so we have a tiny amount of tweets, can we learn anything?

In [225]:
tweets = [t.text for t in tweets]

In [242]:
import re  
try:  
    # UCS-4
    e = re.compile(u'[\U00010000-\U0010ffff]')
except re.error:  
    # UCS-2
    e = re.compile(u'[\uD800-\uDBFF][\uDC00-\uDFFF]')

emojis = []  
for x in tweets:  
    match  = e.search(x)
    if match:
        emojis.append(match.group())

dfe =  pd.DataFrame(emojis,columns=['text'])  
pd.Series(' '.join(dfe['text']).lower().split()).value_counts()[:25]  

🔪    17
😈    17
🐥    17
🙌    17
😃    17
😡    17
🐧    17
🎧    17
📅    17
🌻    17
🖖    17
dtype: int64

In [337]:
wt = [x.split() for x in tweets]

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',
                    level=logging.INFO)
from gensim.models import word2vec
# Set values for various parameters
num_features = 400    # Word vector dimensionality  
min_word_count = 5   # Minimum word count  
num_workers = 4       # Number of threads to run in parallel  
context = 10          # Context window size  
downsampling = 1e-3   # Downsample setting for frequent words

model = word2vec.Word2Vec(wt, workers=num_workers,  
            size=num_features, min_count = min_word_count,
            window = context, sample = downsampling)

model.init_sims(replace=True)  

the model identifies emoji correctly!

In [302]:
print u'\U0001f4c5'


📅


In [303]:
model.most_similar(u'\U0001f4c5',topn=5)  

[(u'calendar', 0.993647038936615),
 (u'Much', 0.9935946464538574),
 (u'confusion', 0.9879370927810669),
 (u'July', 0.9805886745452881),
 (u'last', 0.9793851971626282)]

In [304]:
model.most_similar('calendar',topn=5)

[(u'\U0001f4c5', 0.993647038936615),
 (u'Much', 0.9849915504455566),
 (u'confusion', 0.9778301119804382),
 (u'RT', 0.968165397644043),
 (u'July', 0.9656909704208374)]

In [305]:
model.similarity(u'\U0001f4c5', 'calendar')

0.99364693997127163

In [336]:
s = list(set(emojis))
for i in s:
    try:
        candidates = model.most_similar(i,topn=3)
        for j in candidates:
            try:
                if model.similarity(i, j[0]) > 0.8:
                    print i,j[0]
            except:
                print "?"
                continue
    except KeyError:
        print "need more data"

🐥 🐘
🐥 🐫
🐥 🐦
🖖 🐞
🖖 🐜
🖖 🗻
need more data
📅 calendar
📅 Much
📅 confusion
🎧 http://t.co/e1FQqu4sb1
🎧 Happy
🎧 🐢
🐧 a
😡 THE
😡 emoji
😡 only
need more data
🙌 day…ever?
🙌 Greatest
🙌 https://t.co/1mrqYuDMcv
need more data
🔪 1
🔪 tsunami;
🔪 tsunamis
